In [ ]:
#This script merges cleaned collateral data from an old data freeze, with new data pulled from Axis.
#Inputs:
    #enrollment sheets pulled from AXIS, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs
    #data cleaned and organized from 2020, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/flywheel_data_uploads/data_ready_for_upload/
    #log of collateral IDs pulled from AXIS (https://axis.med.upenn.edu/redcap_v10.3.7/DataExport/index.php?pid=378&report_id=1362) and stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs
    #new data pulled from AXIS (https://axis.med.upenn.edu/redcap_v10.3.7/DataExport/index.php?pid=191&report_id=1318) and stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs
#Outputs:
    #a csv of all collateral battery scales collected for participants enrolled prior to April 1st, 2022 

In [1]:
import pandas as pd

In [2]:
#read in T1 enrollment
axis_t1=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t1.csv',dtype=str)
axis_t1=axis_t1.drop(columns=['scan_1_date'])
#axis_t1

In [3]:
#reformat 
t1_enroll=axis_t1['bblid']
t1_enroll=t1_enroll.tolist()
t1_enroll = [str(t) for t in t1_enroll]

In [4]:
#read in RedCAP IDS from EF tracker
redcap_ids = pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_collateral_ids.csv', dtype=str)
scale_ids = redcap_ids['collateral_redcap_id']
scale_ids = scale_ids.tolist()
scale_ids = [str(i) for i in scale_ids]

In [5]:
#get rid of nan's
scale_ids1=[]
for s in scale_ids:
    if 'nan' not in s:
        #t=s.split('.')[0]
        #print(t)
        scale_ids1.append(s)
print(scale_ids1)

['347', '457', '466', '613', '417', '379', '843', '473', '696', '422', '771', '605', '609', '501', '684', '601', '505', '431', '552', '486', '519', '621', '617', '588', '663', '631', '672', '743', '781', '763', '719', '869', '807', '1006', '859', '854', '887', '892', '992', '1027', '969', '1133', '1129', '996', '1094', '1101', '1047', '1039', '1067', '1112', '1071', '1075', '1083', '1079', '1179', '1268', '1273', '1172', '1237', '1295', '1187', '1248', '1339', '1264', '1307', '1335', '1358', '1372', '1463', '1424', '1483', '2032', '1449', '1387', '2351', '2021', '1528', '2780', '1541', '1833', '2028', '3109', '2283', '2445', '2706', '2497', '3098', '3224', '2813', '3046', '3202', '3206', '2920', '3042', '2947', '2924', '3091', '3303', '3307', '3438', '3381', '3339', '1391812403', '1391812404', '3927', '3991', '3983', '4386', '4216', '4269', '4326', '4330', '4416', '4425', '4273', '4621', '4625', '4538', '1780']


In [6]:
#read in raw data
sr=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_EF_main_scales.csv',dtype=str)

In [7]:
#subset based on scale ids
scales=sr[sr['scales_id'].isin(scale_ids1)]

In [8]:
#merge on SCALE ID, to maintain collateral AND proband bblid 
scales=pd.merge(redcap_ids,scales, left_on="collateral_redcap_id", right_on="scales_id")

In [9]:
#drop, rename and organize columns
scales=scales.rename(columns={"bblid_x": "bblid", "collateralid":"collateral_bblid"})
scales=scales.drop(columns=['collateral_redcap_id','collateral_redcap_id_t2','bblid_y'])

In [12]:
#read in old SR data
cleaned_sr=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/flywheel_data_uploads/data_ready_for_upload/collateralSelfReport_EF_Audit_UPDATED.csv',dtype=str,encoding='latin-1')

In [15]:
#for some reason these don't have scales IDs, so lets separate them from redcap IDs 
ids= cleaned_sr['redcapid']
new_ids=[]
for i in ids:
    x=i[5:]
    new_ids.append(x)

In [16]:
#add new column to dataframe 
cleaned_sr['scales_id']=new_ids

In [17]:
#find who is missing from the cleaned data 
missing=[]
for i in scale_ids1:
    if i not in new_ids:
        missing.append(i)

print(missing)

['2032', '2351', '2021', '2780', '1833', '2028', '3109', '2283', '2445', '2706', '2497', '3098', '3224', '2813', '3046', '3202', '3206', '2920', '3042', '2947', '2924', '3091', '3303', '3307', '3438', '3381', '3339', '1391812403', '1391812404', '3927', '3991', '3983', '4386', '4216', '4269', '4326', '4330', '4416', '4425', '4273', '4621', '4625', '4538', '1780']


In [18]:
#filter out any data that was already organized in the last audit. 
new = scales[scales['scales_id'].isin(missing)]

In [19]:
#drop unnecessary columns
new=new.drop(columns=['date'])
new=new.drop(columns=['admin_proband'])
new=new.drop(columns=['admin_proband_group'])

In [20]:
t1_all_scales = pd.concat([cleaned_sr, new], axis=0, sort=False)

In [21]:
#add a "timepoint" variable (this is t1) and merge with scan IDs so data is easily used w BIDS 
t1_all_scales=pd.merge(t1_all_scales,axis_t1, left_on="bblid", right_on="bblid")
t1_all_scales=t1_all_scales.rename(columns={"scan_id_timepoint_1": "scan_id"})
t1_all_scales['timepoint']= '1'

In [23]:
#and now to add T2 

In [34]:
#read in T1 enrollment
axis_t2=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t2.csv',dtype=str)
axis_t2=axis_t2.drop(columns=['scan_2_date'])
#axis_t2

In [35]:
#reformat 
t2_enroll=axis_t2['bblid']
t2_enroll=t2_enroll.tolist()
t2_enroll = [str(t) for t in t2_enroll]

In [36]:
#get T2 scale ids
scale_ids2 = redcap_ids['collateral_redcap_id_t2']
scale_ids2 = scale_ids2.tolist()
scale_ids2 = [str(i) for i in scale_ids2]

In [37]:
#get rid of nan's
scale_ids2t=[]
for s in scale_ids2:
    if 'nan' not in s:
        #t=s.split('.')[0]
        #print(t)
        scale_ids2t.append(s)
print(scale_ids2t)

[' -', '2471', '3234', '2122', '2475', '2222', '2211', '3198', '2187', '2191', '2324', '2600', '2479', '2320', '2604', '2875', '3020', '4601', '2666', '2651', '2773', '4563', '3459', '3463', '3256', '3857', '4153', '4058', '1391812423', '4302', '3414', '1391812420', '4294', '3758', '4206', '4630', '4482', '4523']


In [38]:
scales2=sr[sr['scales_id'].isin(scale_ids2t)]

In [39]:
#merge on SCALE ID, to maintain collateral AND proband bblid 
scales2=pd.merge(redcap_ids,scales2, left_on="collateral_redcap_id_t2", right_on="scales_id")

In [40]:
#drop, rename and organize columns
scales2=scales2.rename(columns={"bblid_x": "bblid", "collateralid":"collateral_bblid"})
scales2=scales2.drop(columns=['collateral_redcap_id','collateral_redcap_id_t2','bblid_y'])

In [41]:
#drop unnecessary columns
scales2=scales2.drop(columns=['date'])
scales2=scales2.drop(columns=['admin_proband'])
scales2=scales2.drop(columns=['admin_proband_group'])

In [42]:
t2_all_scales=pd.merge(scales2,axis_t2, left_on="bblid", right_on="bblid")
t2_all_scales=t2_all_scales.rename(columns={"scan_id_t2": "scan_id"})
t2_all_scales['timepoint']= '2'

In [43]:
#altogether now! 
all_scales = pd.concat([t1_all_scales, t2_all_scales], axis=0, sort=False)

In [44]:
all_scales.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/outputs/EF_collaterall_scales.csv', sep = ',', index=False)